# CMRM Homework Assignment No. 1 (HW1)

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

# import other libraries
import deeplake
from tqdm import tqdm
import sklearn as sk
import sklearn.preprocessing as skp
import sklearn.svm 
import joblib
import seaborn as sns
from scipy.signal import spectrogram
import IPython.display as ipd


## Question 1

In [ ]:
# Import dataset
dataset = deeplake.load("hub://activeloop/gtzan-genre") #

In [ ]:
Fs = 22050
n_samples = 29 * Fs # duration in samples 

# Define downsampling factors
sub_train = 10
sub_test = 52

# Extract classes
genre_names = ['pop', 'metal', 'classical', 'rock', 'blues', 'jazz', 'hiphop', 'reggae', 'disco', 'country'];

train_indices = list(range(0, 1000, sub_train))
test_indices = list(range(11, 1000, sub_test))

genre_train = dataset['genre'][train_indices]
genre_test = dataset['genre'][test_indices]

print(f"Shapes of genre_train:{np.shape(genre_train)}")
print(f"Shapes of genre_test:{np.shape(genre_test)}")

# Extract training set
audio_train = []
for i in tqdm(train_indices):
    audio = dataset['audio'][i].numpy()
    audio_train.append(audio[:n_samples]) 
audio_train = np.squeeze(np.array(audio_train), axis=-1)

# Extract test set
audio_test = []
for i in tqdm(test_indices):
    audio = dataset['audio'][i].numpy()
    audio_test.append(audio[:n_samples]) 
audio_test = np.squeeze(np.array(audio_test), axis=-1)

print(f"Shapes of audio_train:{np.shape(audio_train)}")
print(f"Shapes of audio_test:{np.shape(audio_test)}")

In [ ]:
# Plot the first wav in the train set
plt.figure(figsize=(5,3))
t = list(range(0,len(audio_train[0])))
t = [x / Fs for x in t]
plt.plot(t, audio_train[0])
plt.title('First audio file')
plt.xlabel('Time [seconds]')
plt.ylabel('Amplitude')
plt.show()
ipd.display(ipd.Audio(audio_train[0], rate=Fs))

## Plot the first wav in the train set


## Question 2

In [ ]:
# Preprocessing
scaler = skp.MinMaxScaler(feature_range=(-1, 1))

for i in range(len(audio_train)):
    audio_train[i] = scaler.fit_transform(audio_train[i].reshape(-1,1)).squeeze()
    
for i in range(len(audio_train)):
    audio_train[i] = scaler.fit_transform(audio_train[i].reshape(-1,1)).squeeze()

In [ ]:
# Plot the first wav in the train set after preprocessing
plt.figure(figsize=(5,3))
plt.plot(t, audio_train[0])
plt.title('First normalized audio file')
plt.xlabel('Time [seconds]')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# Compute local average
def compute_local_average(x, M, Fs):
# M numero di sample per window

    L = len(x)
    local_average = np.zeros(L)

    for i in range(L):
        start = max(0, i - M)
        end = min(L, i + M + 1)
        local_average[i] = np.sum(x[start:end]) * (1/(2*M + 1))
    return local_average
    

# Compute the principal argument
def principal_argument(x):
    
    y = (x + 0.5) % 1 - 0.5
    
    return y

# Compute the Phase-Based Novelty function
def compute_phase_novelty(y=None, Fs=1, N=1024, H=64, M=40, norm=True, plot=False):
    
    # Compute the STFT
    X = librosa.stft(y, n_fft=N, hop_length=H, win_length=N, window='hann')
    
    # Compute the novelty rate
    Fs_nov = Fs / H 
    
    # Extract the phase 
    phase = np.angle(X) / (2*np.pi)
    
    # Compute first and second derivatives of the phase
    phase_der1 = principal_argument(np.diff(phase, axis=1))
    phase_der2 = principal_argument(np.diff(phase_der1, axis=1))
    
    # Accumulation over frequency axis
    nov = np.sum(np.abs(phase_der2), axis=0)
    nov = np.concatenate((nov, np.array([0, 0])))
    
    # Local average subtraction and half-wave rectification
    local_average = compute_local_average(nov, M, Fs)
    nov = nov - local_average
    nov[nov < 0] = 0
    
    # Normalization
    if norm: 
        max_value = np.max(nov)
        if max_value > 0:
            nov = nov / max_value
        
    # Plot
    if plot:
        time_axis = np.arange(nov.shape[0]) / Fs_nov
        
        plt.figure(figsize=(10,4))
        plt.xlim([time_axis[0], time_axis[-1]])
        plt.title('Phase-based novelty function')
        plt.xlabel('Time [seconds]')
        plt.plot(time_axis, nov)
    
    return nov, Fs_nov

In [ ]:
# Test the novelty function on the first wav in the train set
nov, Fs_nov = compute_phase_novelty(audio_train[0], Fs=Fs, norm=1, plot = True) 

## Question 3

In [ ]:
def compute_feature_vector(x, Fs, N=2048, H=128):
    
    # Compute rhythmic features
    nov, Fs_nov = compute_phase_novelty(y=x, Fs=Fs, norm=1, plot=0) #rimetti M = 0!
    novelty_std = np.std(nov)
    novelty_mean = np.mean(nov)
    
    tempogram = librosa.feature.tempogram(y=x, sr=Fs) #prova y = nov
    zero_crossings = librosa.feature.zero_crossing_rate(x)
    zero_crossings_std = np.std(zero_crossings)
    zero_crossings_mean = np.mean(zero_crossings)

    spectral_flux = np.diff(librosa.onset.onset_strength(y=x, sr=Fs))    
    spectral_flux_std = np.std(spectral_flux)
    spectral_flux_mean = np.mean(spectral_flux)
    tempo = librosa.feature.rhythm.tempo(onset_envelope=spectral_flux, y=x, sr=Fs, hop_length=H, tg=tempogram)[0]
    
    novelty_std = novelty_std.reshape((1,))
    novelty_mean = novelty_mean.reshape((1,))
    zero_crossings_std = zero_crossings_std.reshape((1,))
    zero_crossings_mean = zero_crossings_mean.reshape((1,))
    spectral_flux_std = spectral_flux_std.reshape((1,))
    spectral_flux_mean = spectral_flux_mean.reshape((1,))
    tempo = tempo.reshape((1,))

    f_vector = np.concatenate([nov, novelty_std, novelty_mean,
                               zero_crossings.flatten(), zero_crossings_std, zero_crossings_mean,
                               spectral_flux_std, spectral_flux_mean, 
                               tempogram.flatten(), tempo])
    return f_vector

In [ ]:
# Compute feature vector for all the audio files inside the training set
N = 2048
H = 128

train_fvector = []
for i in tqdm(range(100)):
    f_v = compute_feature_vector(audio_train[i], Fs, N, H)
    train_fvector.append(f_v) 
train_fvector = np.array(train_fvector) 
    
test_fvector = []
for i in tqdm(range(20)):
    f_v = compute_feature_vector(audio_test[i], Fs, N, H)
    test_fvector.append(f_v)
test_fvector = np.array(test_fvector)  

In [ ]:
# Check train_fvector and genre_train shapes
print(f"Shapes of train_fvector: {np.shape(train_fvector)}")
print(f"Shapes of test_fvector: {np.shape(test_fvector)}")

## Question 4

In [ ]:
# Define model parameters
C=0.1
kernel = 'poly'
N=train_fvector.shape[0]
H=train_fvector.shape[1]

if not os.path.exists('my_model/'):
    os.mkdir('my_model/')

# Train SVC
model = sklearn.svm.SVC(C=C, kernel=kernel)
model.fit(train_fvector, genre_train.numpy().flatten())

# Save the model
file = f'my_model/svc_{kernel}_C_{C}_N_{N}_H_{H}.joblib'
joblib.dump(model, file)
print(f"Model saved: {file}")
 

In [ ]:
# Print the accuracy on the training set
train_accuracy = sk.metrics.accuracy_score(genre_train, model.predict(train_fvector))
print(f"Accuracy on the training set: {train_accuracy * 100:.2f}%")

## Question 5

In [ ]:
# Classify the test set
predictions = model.predict(test_fvector)

In [ ]:
# Print the accuracy
test_accuracy = sk.metrics.accuracy_score(genre_test, predictions)
print(f"Accuracy on the test set: {test_accuracy * 100:.2f}%")

In [ ]:
# Compute the confusion matrix
cmatrix = sk.metrics.confusion_matrix(genre_test, predictions)

# Plot the confusion matrix
plt.figure(figsize=(8, 8))
sns.heatmap(cmatrix, annot=0, fmt="d", cmap="Blues", cbar=True,
            xticklabels=genre_names,
            yticklabels=genre_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()